In [6]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

# Imports
import numpy as np
import tensorflow as tf
import gzip
import cPickle

In [7]:
def load_zipped_pickle(filename):
    with gzip.open(filename, 'rb') as f:
        loaded_object = cPickle.load(f)
        return loaded_object

In [ ]:
def cnn_model_fn(features, labels, mode):
    """Model function for CNN."""
    # Input Layer
    input_layer = tf.reshape(features["x"], [-1, 64, 64, 3])

    # Convolutional Layer #1
    conv1 = tf.layers.conv2d(inputs=input_layer, filters=32, kernel_size=[5, 5], padding="same", activation=tf.nn.relu)

    # Pooling Layer #1
    pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)

    # Convolutional Layer #2 and Pooling Layer #2
    conv2 = tf.layers.conv2d(inputs=pool1, filters=64, kernel_size=[5, 5], padding="same", activation=tf.nn.relu)
    pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)

    # Dense Layer
    pool2_flat = tf.reshape(pool2, [-1, 16 * 16 * 64])
    dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)
    dropout = tf.layers.dropout(inputs=dense, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)

    # Logits Layer
    logits = tf.layers.dense(inputs=dropout, units=200)

    predictions = {
        # Generate predictions (for PREDICT and EVAL mode)
        "classes": tf.argmax(input=logits, axis=1),
        # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
        # `logging_hook`.
        "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
    }

    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

    # Calculate Loss (for both TRAIN and EVAL modes)
    onehot_labels = tf.one_hot(indices=tf.cast(labels, tf.int32), depth=200)
    loss = tf.losses.softmax_cross_entropy(onehot_labels=onehot_labels, logits=logits)

    # Configure the Training Op (for TRAIN mode)
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1)
        train_op = optimizer.minimize(loss=loss, global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

    # Add evaluation metrics (for EVAL mode)
    eval_metric_ops = {"accuracy": tf.metrics.accuracy(labels=labels, predictions=predictions["classes"])}
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

In [8]:
def cnn_model2_fn(features, labels, mode):
    """Model function for CNN."""
    # Input Layer
    input_layer = tf.reshape(features["x"], [-1, 64, 64, 3])
    
    # Convolutional Layer #1
    conv1 = tf.layers.conv2d(inputs=input_layer, filters=32, kernel_size=[3, 3], padding="same", activation=tf.nn.relu)    
    
    # Convolutional Layer #2
    conv2 = tf.layers.conv2d(inputs=conv1, filters=32, kernel_size=[3, 3], padding="same", activation=tf.nn.relu)
    
    # Pooling Layer #1
    pool1 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)
    
    # dropout #1
    #dropout1 = tf.layers.dropout(inputs=pool1, rate=0.25, training=mode == tf.estimator.ModeKeys.TRAIN)
    dropout1 = tf.layers.dropout(inputs=pool1, rate=0.25)
    
    # Convolutional Layer #3
    conv3 = tf.layers.conv2d(inputs=dropout1, filters=64, kernel_size=[3, 3], padding="same", activation=tf.nn.relu)
    
    # Convolutional Layer #4
    conv4 = tf.layers.conv2d(inputs=conv3, filters=64, kernel_size=[3, 3], padding="same", activation=tf.nn.relu)

    # Pooling Layer #2
    pool2 = tf.layers.max_pooling2d(inputs=conv4, pool_size=[2, 2], strides=2)
    pool2_flat = tf.reshape(pool2, [-1, 16 * 16 * 64])

    # dropout #2
    dropout2 = tf.layers.dropout(inputs=pool2_flat, rate=0.25)
    
    # dense #1
    dense1 = tf.layers.dense(inputs=dropout2, units=256, activation=tf.nn.relu)

    # dropout #3
    dropout3 = tf.layers.dropout(inputs=dense1, rate=0.5)

    # dense #2
    logits = tf.layers.dense(inputs=dropout3, units=200, activation=tf.nn.relu)

    predictions = {
        # Generate predictions (for PREDICT and EVAL mode)
        "classes": tf.argmax(input=logits, axis=1),
        # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
        # `logging_hook`.
        "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
    }

    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

    # Calculate Loss (for both TRAIN and EVAL modes)
    onehot_labels = tf.one_hot(indices=tf.cast(labels, tf.int32), depth=200)
    loss = tf.losses.softmax_cross_entropy(onehot_labels=onehot_labels, logits=logits)

    # Configure the Training Op (for TRAIN mode)
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
        train_op = optimizer.minimize(loss=loss, global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

    # Add evaluation metrics (for EVAL mode)
    eval_metric_ops = {"accuracy": tf.metrics.accuracy(labels=labels, predictions=predictions["classes"])}
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

In [9]:
tf.logging.set_verbosity(tf.logging.INFO)

# load data
print("loading data from file...")
data = load_zipped_pickle("tinyImageData")
train_data = np.array(data["train"]["data"], dtype = np.float16)
train_labels = np.array(data["train"]["target"], dtype = np.float16)
val_data = np.array(data["val"]["data"], dtype = np.float16)
val_labels = np.array(data["val"]["target"], dtype = np.float16)

loading data from file...


In [10]:
# Create the Estimator
tiny_imagenet_clf = tf.estimator.Estimator(model_fn=cnn_model2_fn, model_dir="tiny_imagenet_cnn_model")

# Set up logging for predictions
# Log the values in the "Softmax" tensor with label "probabilities"
tensors_to_log = {"probabilities": "softmax_tensor"}
logging_hook = tf.train.LoggingTensorHook(tensors=tensors_to_log, every_n_iter=500)

# Train the model
train_input_fn = tf.estimator.inputs.numpy_input_fn(x={"x": train_data}, y=train_labels, batch_size=300,
                                                    num_epochs=30, shuffle=True)
tiny_imagenet_clf.train(input_fn=train_input_fn, steps=20000, hooks=[logging_hook])

# Evaluate the model and print results
eval_input_fn = tf.estimator.inputs.numpy_input_fn(x={"x": val_data}, y=val_labels, num_epochs=1, shuffle=False)
eval_results = tiny_imagenet_clf.evaluate(input_fn=eval_input_fn)
print(eval_results)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f002e759490>, '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_num_ps_replicas': 0, '_tf_random_seed': 1, '_master': '', '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 100, '_model_dir': 'tiny_imagenet_cnn_model', '_save_summary_steps': 100}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into tiny_imagenet_cnn_model/model.ckpt.
INFO:tensorflow:probabilities = [[ 0.          0.          0.         ...,  0.          0.          0.        ]
 [ 0.          0.          0.         ...,  0.00000036  0.          0.        ]
 [ 0.          0.          0.         ...,  0.          0.          0.        ]
 ..., 
 [ 0.       

INFO:tensorflow:loss = 5.2617, step = 3501 (5.324 sec)
INFO:tensorflow:global_step/sec: 18.5892
INFO:tensorflow:loss = 5.3281, step = 3601 (5.378 sec)
INFO:tensorflow:global_step/sec: 18.4989
INFO:tensorflow:loss = 5.3086, step = 3701 (5.407 sec)
INFO:tensorflow:global_step/sec: 18.6205
INFO:tensorflow:loss = 5.3203, step = 3801 (5.369 sec)
INFO:tensorflow:global_step/sec: 18.6172
INFO:tensorflow:loss = 5.4766, step = 3901 (5.373 sec)
INFO:tensorflow:global_step/sec: 18.4791
INFO:tensorflow:probabilities = [[ 0.00143719  0.00208282  0.00143719 ...,  0.00176239  0.00257301
   0.00143719]
 [ 0.00113201  0.00213814  0.00113201 ...,  0.00151634  0.00130749
   0.00113201]
 [ 0.00191498  0.00298882  0.00191498 ...,  0.00252342  0.00305367
   0.00191498]
 ..., 
 [ 0.00107574  0.00178623  0.00107574 ...,  0.00107574  0.00143528
   0.00107574]
 [ 0.00095797  0.00178909  0.00095797 ...,  0.00103188  0.00170803
   0.00095797]
 [ 0.00224304  0.0032959   0.00224304 ...,  0.00296402  0.00407791
   0

INFO:tensorflow:loss = 5.3594, step = 7501 (5.404 sec)
INFO:tensorflow:global_step/sec: 18.7895
INFO:tensorflow:loss = 5.293, step = 7601 (5.318 sec)
INFO:tensorflow:global_step/sec: 18.5688
INFO:tensorflow:loss = 4.668, step = 7701 (5.386 sec)
INFO:tensorflow:global_step/sec: 18.632
INFO:tensorflow:loss = 3.6641, step = 7801 (5.367 sec)
INFO:tensorflow:global_step/sec: 18.8347
INFO:tensorflow:loss = 4.9375, step = 7901 (5.309 sec)
INFO:tensorflow:global_step/sec: 18.8378
INFO:tensorflow:probabilities = [[ 0.00488663  0.00565338  0.0055542  ...,  0.00524139  0.0047226
   0.00463486]
 [ 0.00572968  0.0065155   0.00639343 ...,  0.00651932  0.00434494
   0.00434494]
 [ 0.0055809   0.00684738  0.0056572  ...,  0.00607681  0.00439072
   0.00439072]
 ..., 
 [ 0.00493622  0.00537491  0.00501633 ...,  0.00501633  0.004879    0.004879  ]
 [ 0.00509644  0.00771713  0.00603485 ...,  0.008255    0.00381088
   0.00381088]
 [ 0.00492859  0.00519943  0.00500488 ...,  0.00503922  0.00490952
   0.00490

In [ ]:
print(data)